<div style="width: 100%; clear: both;">
<div style="float: left; width: 50%;">
<img src="http://www.uoc.edu/portal/_resources/common/imatges/marca_UOC/UOC_Masterbrand.jpg" align="left">
</div>
<div style="float: right; width: 50%;">
<p style="margin: 0; padding-top: 22px; text-align:right;"><a href="http://www.editorialuoc.cat/mineria-de-datos" target="_blank">Minería de datos: Modelos y algoritmos</a></p>
<p style="margin: 0; text-align:right;">Estudios de Informática, Multimedia y Telecomunicación</p>
<p style="margin: 0; text-align:right; padding-button: 100px;">Universitat Oberta de Catalunya (<b>UOC</b>)</p>
</div>
</div>
<div style="width:100%;">&nbsp;</div>

# CH11EJ1 Máquinas de soporte vectorial

En este ejemplo aprenderemos cómo utilizar las máquinas de soporte vectorial a partir del paquete de R `e1071`.

Trabajaremos los distintos modelos de kernel: lineal, radial, polinomial y sigmoidal. También compararemos los resultados de trabajar con dos dimensiones o con todas.

## 1. Instalar los packages necesarios (en caso de no haberlo hecho anteriormente)

In [7]:
if(!require(e1071)){
    install.packages('e1071', repos='http://cran.es.r-project.org')
    require(e1071)
}
set.seed(17)

## 2. Carga de los datos

In [8]:
dataRaw <- read.table(url("https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/wpbc.data"), 
                  sep=",", header=FALSE)

# Visualizar un resumen de los datos
summary(dataRaw)
head(dataRaw)

       V1          V2            V3               V4              V5       
 Min.   :   8423   N:151   Min.   :  1.00   Min.   :10.95   Min.   :10.38  
 1st Qu.: 855745   R: 47   1st Qu.: 14.00   1st Qu.:15.05   1st Qu.:19.41  
 Median : 886339           Median : 39.50   Median :17.29   Median :21.75  
 Mean   :1990469           Mean   : 46.73   Mean   :17.41   Mean   :22.28  
 3rd Qu.: 927996           3rd Qu.: 72.75   3rd Qu.:19.58   3rd Qu.:24.66  
 Max.   :9411300           Max.   :125.00   Max.   :27.22   Max.   :39.28  
                                                                           
       V6               V7               V8                V9         
 Min.   : 71.90   Min.   : 361.6   Min.   :0.07497   Min.   :0.04605  
 1st Qu.: 98.16   1st Qu.: 702.5   1st Qu.:0.09390   1st Qu.:0.11020  
 Median :113.70   Median : 929.1   Median :0.10190   Median :0.13175  
 Mean   :114.86   Mean   : 970.0   Mean   :0.10268   Mean   :0.14265  
 3rd Qu.:129.65   3rd Qu.:1193.5   3r

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,⋯,V26,V27,V28,V29,V30,V31,V32,V33,V34,V35
,<int>,<fct>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,119513,N,31,18.02,27.60,117.50,1013.0,0.09489,0.1036,0.1086,⋯,139.70,1436.0,0.1195,0.1926,0.3140,0.1170,0.2677,0.08113,5.0,5
2,8423,N,61,17.99,10.38,122.80,1001.0,0.11840,0.2776,0.3001,⋯,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890,3.0,2
3,842517,N,116,21.37,17.44,137.50,1373.0,0.08836,0.1189,0.1255,⋯,159.10,1949.0,0.1188,0.3449,0.3414,0.2032,0.4334,0.09067,2.5,0
4,843483,N,123,11.42,20.38,77.58,386.1,0.14250,0.2839,0.2414,⋯,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,0.17300,2.0,0
5,843584,R,27,20.29,14.34,135.10,1297.0,0.10030,0.1328,0.1980,⋯,152.20,1575.0,0.1374,0.2050,0.4000,0.1625,0.2364,0.07678,3.5,0
6,843786,R,77,12.75,15.29,84.60,502.7,0.11890,0.1569,0.1664,⋯,107.30,733.2,0.1706,0.4196,0.5999,0.1709,0.3485,0.11790,2.5,0


## 3. Generamos juego de datos de pruebas y de entrenamiento

In [9]:
# Separamos los datos de las etiquetas
labels <- dataRaw[,2]
data <- dataRaw[,3:32]

# Seleccionamos el 90% para entrenamiento y el 10% para test
NObs <-  nrow(data)
NTrain <- round(NObs*0.9)
NTest <- NObs - NTrain

# Definir el conjunto de entrenamiento:
train <- data[1:NTrain,]
labelsTrain <- labels[1:NTrain]

# Definir el conjunto de test:
test <- data[(NTrain+1):NObs,]
labelsTest <- labels[(NTrain+1):NObs]

## 4. Proyectamos los datos en las 2 componentes principales

In [10]:
# Proyectamos los datos en las primeras 2 componentes principales
pca <- princomp(train)
train <- predict(pca,train)[,1:2]
test <- predict(pca, test)[,1:2]

## 5. Generamos los modelos de SVM

Emplearemos distintas configuraciones de kernel

### 5.1. Generamos un modelo SVM lineal

In [11]:
# Entrenamos una SVM Lineal
trainFact=data.frame(train,y=as.factor(labelsTrain))
svmfit=svm(y~.,data=trainFact,kernel="linear",cross = 5)

# Predecir el resultado del propio conjunto de entrenamiento
out=predict(svmfit,train)
NTrain
# Porcentaje de acierto
print(sum(out==labelsTrain))/NTrain

# Predecir el resultado usando el conjunto de test
out=predict(svmfit,test)
NTest
# Porcentaje de acierto
print(sum(out==labelsTest))/NTest

[1] 178

[1] 132


[1] 0.741573

[1] 20

[1] 19


[1] 0.95

### 5.2. Generamos un modelo SVM radial

In [12]:
#Función Kernel Radial
svmfit=svm(y~.,data=trainFact,kernel="radial",cross=5)
out=predict(svmfit,test)

# Porcentaje de acierto
print(sum(out==labelsTest))/NTest

[1] 19


[1] 0.95

### 5.3. Generamos un modelo SVM polinomial

In [13]:
#Función Kernel Polinomial
svmfit=svm(y~.,data=trainFact,kernel="polynomial",cross=5)
out=predict(svmfit,test)

# Porcentaje de acierto
print(sum(out==labelsTest))/NTest

[1] 18


[1] 0.9

### 5.4. Generamos un modelo SVM sigmoidal

In [14]:
#Función Kernel Sigmoidal
svmfit=svm(y~.,data=trainFact,kernel="sigmoid",cross=5)
out=predict(svmfit,test)

# Porcentaje de acierto
print(sum(out==labelsTest))/NTest

[1] 17


[1] 0.85

### 5.5. Generamos un modelo SVM lineal con el juego de datos entero

In [15]:
# Aplicaremos SVM con funcion kernel lineal sobre el
# juego de datos entero. Es decir, sin reduccion de 
# dimensiones
train <- data[1:NTrain,]
labelsTrain <- labels[1:NTrain]
test <- data[(NTrain+1):NObs,]
labelsTest <- labels[(NTrain+1):NObs]

trainFact <- data.frame(train, y=as.factor(labelsTrain))
svmfit <- svm(y~., data=trainFact, kernel="linear", cross=5)
print(svmfit)

out <- predict(svmfit, train)

# Porcentaje de acierto
print(sum(out==labelsTrain))/NTrain


Call:
svm(formula = y ~ ., data = trainFact, kernel = "linear", cross = 5)


Parameters:
   SVM-Type:  C-classification 
 SVM-Kernel:  linear 
       cost:  1 

Number of Support Vectors:  83

[1] 159


[1] 0.8932584